In [15]:
import cv2
import numpy as np
import time

net = cv2.dnn.readNet('runs/detect/train18/weights/best.onnx')
image = cv2.imread('head/image_20240819_104304.png')

## 运行 YOLO 模型并可视化结果

In [16]:
start_time = time.time()

# 获取图片的高度和宽度
height, width = image.shape[:2]

input_size = (320, 320)

# 预处理图片
blob = cv2.dnn.blobFromImage(image, 1/255.0, input_size, swapRB=True, crop=False)
net.setInput(blob)

# 推理
outputs = net.forward()

# 后处理，提取检测结果
boxes = []
confidences = []
class_ids = []

outputs = outputs[0].transpose(1, 0)
print(outputs.shape)


for detection in outputs:
    scores = detection[4:]
    x, y, w, h = detection[:4]
    classId = np.argmax(scores)
    confidence = scores[classId]

    if confidence > 0.5:
        # 计算边界框
        x = int((x - w / 2) / input_size[0] * width)
        y = int((y - h / 2) / input_size[1] * height)
        w = int(w / input_size[0] * width)
        h = int(h / input_size[1] * height)

        boxes.append([x, y, w, h])
        confidences.append(float(confidence))
        class_ids.append(classId)

end_time = time.time()

# 应用非最大抑制 (NMS) 来抑制重叠框
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.6)

print(f"time: {end_time - start_time}")
classes = ['green', 'orange']
image2 = image.copy()

# 画出检测框
for i in indices:
    box = boxes[i]
    x, y, w, h = box[0], box[1], box[2], box[3]

    label = f"{classes[class_ids[i]]}: {confidences[i]:.2f}"
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    print(x, y, w, h, class_ids[i], confidences[i])
    cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# 显示结果图片
cv2.imshow('Original Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

(2100, 6)
time: 0.020028352737426758
150 366 204 113 1 0.9031565189361572
316 353 144 125 0 0.7293785214424133


## 选择距离机器人最近的方块

In [17]:
boxes = [boxes[i] for i in indices]
dis = [(i, (x + w // 2 - 320) ** 2 + (480 - y - h) ** 2) for i, (x, y, w, h) in enumerate(boxes)]
dis_sorted = sorted(dis, key=lambda x: x[1])
box_x, box_y, box_w, box_h = boxes[dis_sorted[0][0]]
cv2.rectangle(image2, (box_x, box_y), (box_x + box_w, box_y + box_h), (0, 255, 0), 2)
cv2.imshow("Final Image", image2)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 检测方块是否成功搬起

In [18]:
# 不同色块的hsv范围
color_range = {
    'green': [(45, 40, 60), (51, 255, 255)],
    'orange': [(8, 120, 120), (18, 255, 255)]
}
img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
mask1 = cv2.inRange(img, color_range['green'][0], color_range['green'][1])
mask2 = cv2.inRange(img, color_range['orange'][0], color_range['orange'][1])
mask = cv2.bitwise_or(mask1, mask2)
mask_opened = cv2.dilate(mask, np.ones((4, 4), np.uint8), iterations=2)
mask = np.zeros_like(mask)
mask[430:480, 240:440] = mask_opened[430:480, 240:440]
ratio = cv2.countNonZero(mask) / 50 / 200
print(ratio)
cv2.imshow("Image", image)
cv2.imshow("Mask", mask_opened)
cv2.imshow("Mask Crop", mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

0.6965
